In [1]:
import pandas as pd

github_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv"

url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [2]:
df = df.iloc[:300]

In [ ]:
from sentence_transformers import SentenceTransformer

model_name = "multi-qa-mpnet-base-dot-v1"
embedding_model = SentenceTransformer(model_name)

In [4]:
answer_llm = df.iloc[0].answer_llm
encoded_answer_llm = embedding_model.encode(answer_llm)
encoded_answer_llm[0]

np.float32(-0.42244658)

In [5]:
answer_llm_embeddings = df['answer_llm'].apply(lambda x: embedding_model.encode(x))
original_llm_embeddings = df['answer_orig'].apply(lambda x: embedding_model.encode(x))

In [6]:
import numpy as np
dot_products = [np.dot(text_emb, orig_emb) for text_emb, orig_emb in zip(answer_llm_embeddings, original_llm_embeddings)]

In [7]:
np.percentile(dot_products, 75)

np.float32(31.674305)

In [8]:
def normalize_vector(vector):
    norm = np.linalg.norm(vector)
    return vector / norm# if norm != 0 else vector

np.linalg.norm(answer_llm_embeddings[0])

normalized_texts_embeddings = [normalize_vector(embedding) for embedding in answer_llm_embeddings]
normalized_original_embeddings = [normalize_vector(embedding) for embedding in original_llm_embeddings]

dot_products = [np.dot(text_emb, orig_emb) for text_emb, orig_emb in zip(normalized_texts_embeddings, normalized_original_embeddings)]
np.percentile(dot_products, 75)

np.float32(0.8362348)

In [9]:
from rouge import Rouge
rouge_scorer = Rouge()

r = df.iloc[10]
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [10]:
np.mean([scores['rouge-1']['f'], scores['rouge-2']['f'],scores['rouge-l']['f']]
)

np.float64(0.35490034990035496)

In [11]:
rougetwo = []
for i, r in df.iterrows():
    scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
    rougetwo.append({
        "r": scores['rouge-2']['r'],
        "p": scores['rouge-2']['p'],
        "f": scores['rouge-2']['f'],
    })

In [12]:
roguetwodf = pd.DataFrame(rougetwo) 

In [13]:
roguetwodf.mean()

r    0.198613
p    0.258626
f    0.206965
dtype: float64